In [1]:
dictionary = {
    "北京烤鸭",
    "特别",
    "喜欢"
}
sentence = "他特别喜欢北京烤鸭"

In [248]:
# 最大后向匹配（递归）
def seg(sentence):
    res = []
    def maxmatch(res, sentence, dictionary):
        if len(sentence) == 0:
            return list()
        for i in reversed(range(1, len(sentence)+1)):
            fw = sentence[:i]
            rw = sentence[i:]
            if fw in dictionary:
                res.append(fw)
                return maxmatch(res, rw, dictionary)
        res.append(fw)
        return maxmatch(res, rw, dictionary)
    maxmatch(res, sentence, dictionary)
    return res

In [3]:
seg(sentence)

['他', '特别', '喜欢', '北京烤鸭']

# 分词

参考资料：

- [ysc/word: Java 分布式中文分词组件 - word 分词](https://github.com/ysc/word)

## 词典

In [234]:
import pnlp

In [235]:
dictionary = pnlp.read_lines("./dic.txt")
# 字典中不能包括一个字的词，否则都被切成一个字了
dictionary = [w for w in dictionary if len(w) > 1]
dictionary = set(dictionary)
len(dictionary)

415870

In [236]:
MAX_LEN = max(len(w) for w in dictionary)
MAX_LEN

16

## 最大正向匹配

In [162]:
def forward_max_match2(sent):
    res = []
    while len(sent) > 0:
        length = min(MAX_LEN, len(sent))
        try_word = sent[:length]
        while try_word not in dictionary:
            if len(try_word) == 1:
                break
            try_word = try_word[:-1]
        res.append(try_word)
        sent = sent[len(try_word):]
    return res

In [163]:
def forward_max_match(sent):
    res = []
    length = MAX_LEN
    text_len = len(sent)
    start = 0
    while start < text_len:
        length = min(length, text_len - start)
        while sent[start:start+length] not in dictionary:
            if length == 1:
                break
            length -= 1
        res.append(sent[start:start+length])
        start += length
        length = MAX_LEN
    return res

In [164]:
sen = "美国加州大学的科学家发现"

In [165]:
forward_max_match(sen)

['美国加州大学', '的', '科学家', '发现']

In [166]:
forward_max_match2(sen)

['美国加州大学', '的', '科学家', '发现']

In [171]:
%timeit forward_max_match(sen)

8.67 µs ± 397 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [172]:
%timeit forward_max_match2(sen)

8.51 µs ± 188 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## 正向最小匹配

In [157]:
def forward_min_match(sent):
    res = []
    length = 1
    text_len = len(sent)
    start = 0
    while start < text_len:
        while sent[start:start+length] not in dictionary:
            if length == MAX_LEN or length == text_len - start:
                length = 1
                break
            length += 1
        res.append(sent[start:start+length])
        start += length
        length = 1
    return res

In [158]:
sen = "美国加州大学的科学家发现"

In [159]:
forward_min_match(sen)

['美国', '加州', '大学', '的', '科学', '家', '发现']

## 最大反向匹配

In [225]:
def backward_max_match(sent):
    res = []
    length = MAX_LEN
    text_len = len(sent)
    start = max(0, text_len - length)
    length = min(length, text_len - start)
    while start >= 0 and length > 0:
        while sent[start: start+length] not in dictionary:
            if length == 1:
                break
            length -= 1
            start += 1
        res.append(sent[start: start+length])
        length = MAX_LEN
        if length > start:
            length = start
        start -= length
    return list(reversed(res))

In [226]:
MAX_LEN = max(len(w) for w in dictionary)

In [227]:
sen = "研究生命的起源"
len(sen), MAX_LEN

(7, 16)

In [228]:
backward_max_match(sen)

['研究', '生命', '的', '起源']

## 最小反向匹配

In [237]:
def backward_min_match(sent):
    res = []
    length = 1
    text_len = len(sent)
    start = text_len - length
    while start >= 0:
        while sent[start: start+length] not in dictionary:
            length += 1
            start -= 1
            if length == MAX_LEN or start < 0:
                start += length - 1
                length = 1
                break
        res.append(sent[start: start+length])
        start -= 1
        length = 1
    return list(reversed(res))

In [243]:
sen = "美国加州大学的科学家发现"

In [244]:
backward_min_match(sen)

['美国', '加州', '大学', '的', '科', '学家', '发现']

## 数据对比

In [253]:
sens = [
    "中华人民共和国万岁万岁万万岁",
    "乔布斯是 Apple 产品的设计者",
    "美国加州大学的科学家发现",
    "研究生命的起源",
    "长春市长春节致辞",
    "他从马上下来",
    "有意见分歧"
]

In [254]:
for sen in sens:
    
    res = forward_max_match(sen)
    print("最大前向匹配：", res)
    
    res = backward_max_match(sen)
    print("最大后向匹配：", res)
    
    res = forward_min_match(sen)
    print("最小前向匹配：", res)
    
    res = backward_min_match(sen)
    print("最小后向匹配：", res)
    
    print()

最大前向匹配： ['中华人民共和国', '万岁', '万岁', '万万岁']
最大后向匹配： ['中华人民共和国', '万岁', '万岁', '万万岁']
最小前向匹配： ['中华', '人民', '共和', '国', '万岁', '万岁', '万万', '岁']
最小后向匹配： ['中华', '人民', '共和国', '万岁', '万岁', '万', '万岁']

最大前向匹配： ['乔布斯', '是', ' ', 'A', 'p', 'p', 'l', 'e', ' ', '产品', '的', '设计者']
最大后向匹配： ['乔布斯', '是', ' ', 'A', 'p', 'p', 'l', 'e', ' ', '产品', '的', '设计者']
最小前向匹配： ['乔布', '斯', '是', ' ', 'A', 'p', 'p', 'l', 'e', ' ', '产品', '的', '设计', '者']
最小后向匹配： ['乔', '布斯', '是', ' ', 'A', 'p', 'p', 'l', 'e', ' ', '产品', '的', '设计者']

最大前向匹配： ['美国加州大学', '的', '科学家', '发现']
最大后向匹配： ['美国加州大学', '的', '科学家', '发现']
最小前向匹配： ['美国', '加州', '大学', '的', '科学', '家', '发现']
最小后向匹配： ['美国', '加州', '大学', '的', '科', '学家', '发现']

最大前向匹配： ['研究生', '命', '的', '起源']
最大后向匹配： ['研究', '生命', '的', '起源']
最小前向匹配： ['研究', '生命', '的', '起源']
最小后向匹配： ['研究', '生命', '的', '起源']

最大前向匹配： ['长春市', '长春', '节', '致辞']
最大后向匹配： ['长春', '市长', '春节', '致辞']
最小前向匹配： ['长春', '市长', '春节', '致辞']
最小后向匹配： ['长春', '市长', '春节', '致辞']

最大前向匹配： ['他', '从', '马上', '下来']
最大后向匹配： ['他', '从', '马上', '下来']
最小前向匹配： [